In [1]:

from datetime import datetime

########### Basic Parameters for Running: ################################

DateNow = datetime.today().strftime('%Y%m%d')
TimeNow = datetime.today().strftime('%H%M%S')
    
TFliteNamingAndVersion = "effnet100-" + DateNow + "-" + TimeNow     # Used for tflite Filename
#TFliteNamingAndVersion = "effnet"     # Used for tflite Filename
Load_Model = "eff100-gray.h5"  
ziffer_data_url="images"
output_dir="."
use_grayscale = False
nb_classes = 100
##########################################################################


In [2]:
from b2n.models.prune_quantize import prune, quantization_default
from b2n.data.ziffer import ziffer_data
from b2n.encodings.class_encoding import class_encoding
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


import tensorflow.keras as keras

xz_data, yz_data, fz_data = ziffer_data(ziffer_data_url, use_grayscale=use_grayscale)
x_data, y_data = shuffle(xz_data, yz_data)
y_data = class_encoding(y_data, nb_classes)
print(y_data.shape)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
print(y_train.shape, y_test.shape)

model = keras.models.load_model(Load_Model)

# prune the model
model_pruned = prune(model=model,
        x_train=x_train, 
        y_train=y_train,
        x_test=x_test, 
        y_test=y_test)

# quanitize and save the model
tflite_model = quantization_default(model=model_pruned, 
                            x_train=x_test)
# save the model
filename=TFliteNamingAndVersion + "-q.tflite"                                                     
open( filename, "wb").write(tflite_model)
